In [1]:
import torch
import numpy as np
import open3d as o3d
from tqdm import tqdm
import point_cloud_utils as pcu


def extract_visible_points(points, x_v, y_v, z_v, r):
    num_complete = points.shape[0]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    cam_pos = [x_v, y_v, z_v]
    _, pt_map = pcd.hidden_point_removal(cam_pos, r)
    pcd_vis = pcd.select_by_index(pt_map)
    points_vis = np.asarray(pcd_vis.points).astype(np.float32)
    indices_vis = np.asarray(pt_map)
    return indices_vis, points_vis


def save_pc_as_ply(save_path, points, colors=None, normals=None):
    assert save_path[-3:] == 'ply', 'not .ply file'
    if type(points) == torch.Tensor:
        points = np.asarray(points.detach().cpu())
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    if colors is not None:
        if type(colors) == torch.Tensor:
            colors = np.asarray(colors.detach().cpu())
        assert colors.min()>=0 and colors.max()<=1
        pcd.colors = o3d.utility.Vector3dVector(colors) # should be within the range of [0, 1]
    if normals is not None:
        if type(normals) == torch.Tensor:
            normals = np.asarray(normals.detach().cpu())
        pcd.normals = o3d.utility.Vector3dVector(normals)
    o3d.io.write_point_cloud(save_path, pcd, write_ascii=True) # should be saved as .ply file


def bounding_box_normalization(pc):
    # pc: (num_points, num_channels)
    # pc_normalized: (num_points, num_channels)
    num_points, num_channels = pc.shape
    xyz = pc[:, 0:3]
    attr = pc[:, 3:]
    xyz = xyz - (np.min(xyz, axis=0) + np.max(xyz, axis=0))/2
    max_d = np.max(np.sqrt(np.abs(np.sum(xyz**2, axis=1)))) # a scalar
    xyz_normalized = xyz / max_d
    pc_normalized = np.concatenate((xyz_normalized, attr), axis=1)
    return pc_normalized

def farthest_point_sample(point, npoint=8192):
    """
    Input:
        xyz: pointcloud data, [N, D]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [npoint, D]
    """
    N, D = point.shape
    xyz = point[:,:3]
    centroids = np.zeros((npoint,))
    distance = np.ones((N,)) * 1e10
    farthest = np.random.randint(0, N)
    for i in range(npoint):
        centroids[i] = farthest
        centroid = xyz[farthest, :]
        dist = np.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = np.argmax(distance, -1)
    return centroids.astype(np.int32)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import os,sys
allcarfolders = os.listdir('/mnt/d/junch_data/diffusion_231019/all_chair_30')
allcarfoldersnew=[]
for r in allcarfolders:
    allcarfoldersnew.append(r.split('_')[0])
allcarfoldersnew = sorted(allcarfoldersnew)

In [3]:
import pymeshlab,os
# allfiles = os.listdir('/mnt/d/junch_data/AtlasNet/02958343')
for i in range(0, 100):
    fp = allcarfoldersnew[i]
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(os.path.join('/mnt/d/junch_data/AtlasNet/03001627',fp,'models/model_normalized.obj'))
    ms.load_filter_script('/mnt/d/junch_data/AtlasNet/proc.mlx')
    ms.apply_filter_script()
    ms.save_current_mesh('result1.ply')
    pcd = o3d.io.read_point_cloud("result1.ply")
    xyz = np.asarray(pcd.points)
    rgb = np.asarray(pcd.colors)

    xyz = bounding_box_normalization(xyz.astype(np.float32))
    rgb = rgb[:, 0:3].astype(np.float32)
    cam_pos = np.loadtxt('/mnt/d/junch_data/AtlasNet/cam_pos.xyz').astype(np.float32)

    r = xyz.shape[0] * 0.005

    vis_idx = []
    for cid in tqdm(range(cam_pos.shape[0])):
        x_v, y_v, z_v = cam_pos[cid, 0], cam_pos[cid, 1], cam_pos[cid, 2]
        vis_idx.append(extract_visible_points(xyz, x_v, y_v, z_v, r)[0])
    vis_idx = list(set(np.concatenate(vis_idx)))
    print('input points: {}, selected points: {}'.format(xyz.shape[0], len(vis_idx)))
    input_points = np.concatenate((xyz, rgb), axis=-1)
    output_points = input_points[vis_idx]
    save_pc_as_ply('p_all1.ply', input_points[:, 0:3], colors=input_points[:, 3:6])
    save_pc_as_ply('p_vis1.ply', output_points[:, 0:3], colors=output_points[:, 3:6])
    
    pcd=o3d.io.read_point_cloud("p_vis1.ply")
    p=np.asarray(pcd.points)
    idxes = farthest_point_sample(point=p)
    
    p=np.asarray(pcd.points)[idxes]
    c=np.asarray(pcd.colors)[idxes]
    s_p=os.path.join('/mnt/d/junch_data/diffusion_231019/all_chair_pointcloud_vis',fp+'.ply')
    save_pc_as_ply(s_p,p,c)

FilterScript
Reading filter with name Transfer Color: Face to Vertex
Reading filter with name Poisson-disk Sampling
Reading filter with name Vertex Attribute Transfer
100%|██████████| 82/82 [00:22<00:00,  3.69it/s]


input points: 124475, selected points: 122153


FilterScript
Reading filter with name Transfer Color: Face to Vertex
Reading filter with name Poisson-disk Sampling
Reading filter with name Vertex Attribute Transfer
100%|██████████| 82/82 [00:12<00:00,  6.52it/s]


input points: 91093, selected points: 84379


FilterScript
Reading filter with name Transfer Color: Face to Vertex
Reading filter with name Poisson-disk Sampling
Reading filter with name Vertex Attribute Transfer
100%|██████████| 82/82 [00:17<00:00,  4.75it/s]


...

input points: 90206, selected points: 80657


FilterScript
Reading filter with name Transfer Color: Face to Vertex
Reading filter with name Poisson-disk Sampling
Reading filter with name Vertex Attribute Transfer
100%|██████████| 82/82 [00:25<00:00,  3.19it/s]


input points: 88156, selected points: 72672


FilterScript
Reading filter with name Transfer Color: Face to Vertex
Reading filter with name Poisson-disk Sampling
Reading filter with name Vertex Attribute Transfer
100%|██████████| 82/82 [00:22<00:00,  3.58it/s]


input points: 91293, selected points: 77781
